# Install dependencies

In [ ]:
import torch
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())

# Setting maximum sequence length and quanitzation

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # max we could fit on 1 GPU
dtype = float
load_in_4bit = True # 4bit quantization

# Load base model and tokenizer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Andrei481/llama3-8b-corpus-ro-8k-16b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = ""
)

# Add LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

# Data Preprocessing

In [ ]:
prompt = """Mai jos este prezentată o instrucțiune care descrie o sarcină, însoțită de o intrare care oferă un context suplimentar. Scrieți un răspuns care să completeze în mod corespunzător cererea. Dacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie să răspundeți doar în limba română.

### Instrucțiune:
{}

### Intrare:
{}

### Răspuns:
{}"""

llama3_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sunteți un asistent util, respectuos și onest. Dacă o întrebare nu are niciun sens sau nu este coerentă din punct de vedere factual, explicați de ce în loc să răspundeți la ceva incorect. Dacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie sa răspundeți doar în limba română.<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def format_prompt_chat_llama3(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is not None:
            text = llama3_prompt.format(instruction + " " + input, output) + EOS_TOKEN # IMPORTANT - WITHOUT THIS, GENERATION WILL GO ON FOREVER
        else:
            text = llama3_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

In [ ]:
EOS_TOKEN = tokenizer.eos_token
BOS_TOKEN = tokenizer.bos_token
PAD_TOKEN = tokenizer.pad_token
print(EOS_TOKEN)
print(BOS_TOKEN)
print(PAD_TOKEN)

In [ ]:
dataset = dataset.map(format_prompt_chat_llama3, batched=True)

In [ ]:
dataset["text"][5]

# Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=3,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

# Inference

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    llama3_prompt.format(
        "Care sunt condițiile necesare pentru a avea un deadlock în programare?", # instruction
        "", # input (optional)
        "", # output (leave empty)
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

# Save finetuned model locally

In [ ]:
model.save_pretrained("llama-3-8b-unsloth-corpus-hakurei-ro-lora-adapters")

# Save finetuned to HuggingFace

In [ ]:
if True: model.save_pretrained_merged("llama-3-8b-unsloth-corpus-hakurei-ro-16b", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("Andrei481/llama-3-8b-unsloth-corpus-hakurei-ro-16b", tokenizer, save_method = "merged_16bit", token = "", private=True)